In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
with open("../data/raw/epfl.json") as file:
    epfl = json.load(file)
    
with open("../data/raw/epfl_master_specs.json") as file:
    epfl_masterspecs = json.load(file)

Snapshots from the json objects:

In [3]:
epfl['master']['Data Science']['COM-480']

{'name': 'Data visualization',
 'section': 'SC',
 'language': 'English',
 'note': '',
 'path': '/coursebook/en/data-visualization-COM-480?cb_cycle=bama_cyclemaster&cb_section=sc_ds',
 'semester': 'Spring',
 'exam_form': 'During the semester',
 'credits': '4',
 'subject_examined': 'Data visualization',
 'lecture': '2 Hour(s) per week x 14 weeks',
 'project': '2 Hour(s) per week x 14 weeks',
 'lecturers': [['Vuillon Laurent Gilles Marie',
   'http://people.epfl.ch/319954']],
 'required': ['CS-305 Software engineering (BA)',
  'CS-250 Algorithms (BA)',
  'CS-401 Applied data analysis (MA)'],
 'recommended': ['EE-558 A Network Tour of Data Science (MA)',
  'CS-486 Human computer interaction (MA)',
  'CS-210 Functional programming (BA)'],
 'concepts': ["Being autonomous is a prerequisite, we don't offer office hours and we won't have enough teaching assistants (you've been warned!).",
  'Knowledge of one of the following progrmaming language such as C++, Python, Scala.',
  'Familiarity with

In [4]:
epfl_masterspecs['Civil Engineering']

{'spec_key': {'b': 'Geotechnical Engineering',
  'c': 'Transportation',
  'd': 'Structural Engineering',
  'e': 'Hydraulic Engineering and Energy'},
 'courses': {'CIVIL-597': [],
  'CIVIL-599': [],
  'ENG-445': [],
  'CIVIL-420': ['d'],
  'CIVIL-457': ['c'],
  'CIVIL-402': ['b'],
  'CIVIL-412': ['e'],
  'CIVIL-474': [],
  'CIVIL-491': [],
  'CIVIL-490': [],
  'PENS-490': [],
  'CIVIL-492': [],
  'PENS-491': [],
  'CIVIL-598': [],
  'CIVIL-443': ['d'],
  'CIVIL-422': ['b', 'd'],
  'CIVIL-414': ['d'],
  'CIVIL-435': ['d'],
  'CIVIL-476': ['d'],
  'ENV-508': [],
  'AR-535': ['d'],
  'CIVIL-423': ['b'],
  'CIVIL-430': ['d'],
  'CIVIL-525': ['d'],
  'ENG-470': ['b', 'd', 'e'],
  'ENG-471': ['b', 'd', 'e'],
  'CIVIL-411': ['b', 'd', 'e'],
  'CIVIL-557': ['c'],
  'CIVIL-459': ['c'],
  'ENG-466': [],
  'CIVIL-444': ['b', 'd', 'e'],
  'CIVIL-428': ['b'],
  'CIVIL-436': ['b', 'd', 'e'],
  'CIVIL-437': ['b', 'd', 'e'],
  'CIVIL-515': ['e'],
  'CIVIL-410': ['b', 'e'],
  'CIVIL-403': ['b', 'e'],
  

Let's start with noting that only 25 master's programs are currently offered at EPFL and only 13 bachelor programs. So, we have some outdated programs in our database. Let's remove them.

In [5]:
{level: len(programs) for level, programs in epfl.items()}

{'propedeutics': 14,
 'bachelor': 17,
 'master': 29,
 'minor': 20,
 'doctoral_school': 22}

In [6]:
# Only 13 bachelor programs are currently offered
# + Humanities and Social Sciences Program
# + Design Together ENAC
# https://www.epfl.ch/education/bachelor/programs/
bachelor_not_current = ['Chemistry', 'Chemical Engineering']

# Only 25 master's programs are currently offered
# + Humanities and Social Sciences Program
# https://www.epfl.ch/education/master/programs/
master_not_current = [
    'Bioengineering',
    'Life Sciences and Technologies - master program',
    'Micro- and Nanotechnologies for Integrated Systems'
]

# The following minors are missing, among possibly others:
#   Computational science and engineering
#   Mechanical engineering
# We do not take any action for now

In [7]:
# remove programs not currently offered from the main json object
for level_name in list(epfl.keys()):
    for program_name in list(epfl[level_name].keys()):
        if (
            (level_name == 'bachelor' and program_name in bachelor_not_current) or 
            (level_name == 'master' and program_name in master_not_current)
        ):
            del epfl[level_name][program_name]
            if level_name == 'master' and program_name in epfl_masterspecs:
                # remove program from the list of programs with specializations
                del epfl_masterspecs[program_name]

Let's now analyze master's specializations (no specializations at other levels) and evaluate data accuracy. We compare the data source (studyplan pages) with the studyplan brochures. We have created a separate json object `epfl_master_specs` to store information about specializations.

In [8]:
"""
The studyplan page sometimes differs greatly from the up-to-date studyplan brochure
which might indicate that the studyplan page hasn't been updated.
We skip these specializations for now (we would have to manually type them in):

Architecture
https://www.epfl.ch/education/master/wp-content/uploads/2018/08/ENAC_ARCHI_MA-1.pdf
vs.
https://edu.epfl.ch/studyplan/en/master/architecture
"""
specs_to_remove = ['Architecture']

"""
The specializations legend on studyplan pages is sometimes obsolete (hasn't been removed)
We remove these programs from the list of specialization programs

Materials Science and Engineering (only one specialization now)
https://www.epfl.ch/education/master/wp-content/uploads/2018/08/STI_MX_MA-1.pdf
vs.
https://edu.epfl.ch/studyplan/en/master/materials-science-and-engineering

Microengineering
https://www.epfl.ch/education/master/wp-content/uploads/2018/08/STI_SMT_MA_RV-1.pdf
vs.
https://edu.epfl.ch/studyplan/en/master/microengineering
"""
specs_to_remove.extend(['Materials Science and Engineering', 'Microengineering'])

"""
The specializations legend on studyplan pages is sometimes obsolete, but the studyplan itself
contains references to specializations that correctly correspond to the the studyplan brochure
In this case, we update the legend manually and fix the data

Electrical and Electronics Engineering
https://www.epfl.ch/education/master/wp-content/uploads/2018/08/STI_EL_MA-1.pdf
vs.
https://edu.epfl.ch/studyplan/en/master/electrical-and-electronics-engineering
"""
electrical_electronics_eng_specs = {
    "a": "Microelectronics circuits and systems",
    "b": "Electronic technologies and device-circuit interactions",
    "c": "Bioelectronics",
    "d": "Internet of Things (IoT)",
    "e": "Data Science and Systems",
    "f": "Signal, Image, Video and Communication",
    "g": "Wireless and Photonics Circuits and Systems",
}

We remove all programs in `specs_to_remove`:

In [9]:
for program in specs_to_remove:
    del epfl_masterspecs[program]

... and fix "Electrical and Electronics Engineering" specializations key:

In [10]:
epfl_masterspecs['Electrical and Electronics Engineering']['spec_key'] = electrical_electronics_eng_specs

We observe that some courses have listed specializations not present in the `spec_key` for the given program. We remove these references:

In [11]:
for program_name, program_spec in epfl_masterspecs.items():
    for code, course_spec in program_spec['courses'].items():
        if (any(spec not in program_spec['spec_key'] for spec in course_spec)):
            print(f"{program_name}, spec key: {list(program_spec['spec_key'].keys())}, {code}: {course_spec}")
            
            new_course_spec = [spec for spec in course_spec if spec in program_spec['spec_key']]
            epfl_masterspecs[program_name]['courses'][code] = new_course_spec
            
            print(f"Course spec changed from {course_spec} to {new_course_spec}")

Electrical and Electronics Engineering, spec key: ['a', 'b', 'c', 'd', 'e', 'f', 'g'], EE-588: ['h']
Course spec changed from ['h'] to []
Electrical and Electronics Engineering, spec key: ['a', 'b', 'c', 'd', 'e', 'f', 'g'], MATH-261: ['d', 'e', 'h']
Course spec changed from ['d', 'e', 'h'] to ['d', 'e']
Electrical and Electronics Engineering, spec key: ['a', 'b', 'c', 'd', 'e', 'f', 'g'], CS-423: ['d', 'f', 'h']
Course spec changed from ['d', 'f', 'h'] to ['d', 'f']
Electrical and Electronics Engineering, spec key: ['a', 'b', 'c', 'd', 'e', 'f', 'g'], EE-576: ['h']
Course spec changed from ['h'] to []
Electrical and Electronics Engineering, spec key: ['a', 'b', 'c', 'd', 'e', 'f', 'g'], ME-409: ['h']
Course spec changed from ['h'] to []
Electrical and Electronics Engineering, spec key: ['a', 'b', 'c', 'd', 'e', 'f', 'g'], EE-466: ['h']
Course spec changed from ['h'] to []
Electrical and Electronics Engineering, spec key: ['a', 'b', 'c', 'd', 'e', 'f', 'g'], MICRO-565: ['h']
Course spe

Next, we add a `specs` dictionary to each program in `epfl_masterspecs` with keys and values switching roles compared to the `courses` property. This will make it easier to look up all courses with a given specialization.

In [12]:
# courses property
list(epfl_masterspecs['Computer Science']['courses'].items())[:10]

[('CS-595', []),
 ('CS-599', []),
 ('CS-450', ['b', 'c', 'd', 'e', 'i']),
 ('CS-470', ['a', 'd', 'g']),
 ('CS-442', ['f']),
 ('COM-401', ['d', 'e', 'g', 'j']),
 ('CS-422', ['b', 'c', 'g', 'j']),
 ('CS-438', ['g']),
 ('CS-451', ['c', 'g', 'i', 'j']),
 ('CS-452', ['c', 'g'])]

In [13]:
for program_name, program_spec in epfl_masterspecs.items():
    # initialize dictionary with empty list values
    epfl_masterspecs[program_name]['specs'] = {k: [] for k in epfl_masterspecs[program_name]['spec_key']}
    
    for course_code, course_spec in program_spec['courses'].items():
        # iterate over all specialization codes for each course, and add the course code to the specs dictionary
        for k in course_spec:
            epfl_masterspecs[program_name]['specs'][k].append(course_code)

In [14]:
{'g': epfl_masterspecs['Computer Science']['specs']['g']}

{'g': ['CS-470',
  'COM-401',
  'CS-422',
  'CS-438',
  'CS-451',
  'CS-452',
  'COM-407',
  'CS-420',
  'CS-471',
  'CS-453',
  'COM-405',
  'COM-503',
  'CS-522']}

Now, we have processed the specializations and can move on.

Next step is to create two new json objects by transforming the `epfl` object. The goal will be to remove redundancies in the original object, since it has many courses occuring repeatedly, and not only enable easy access to programs, but also _individual courses_. The idea is to create one object `epfl_courses` with all course details and another object `epfl_programs` which will only include course codes for each program.

In [15]:
epfl_courses = [
    # for each program: ([code1, code2, ...], [course1, course2, ...])
    ([*program.keys()], [*program.values()])
    for level_name, level in epfl.items()
    for program_name, program in level.items()
]

In [16]:
# all courses: [[code1, code2, ...], [course1, course2, ...]]
epfl_courses = list(
    map(
        lambda x: sum(x, []),
        zip(*epfl_courses)
    )
)

Let's analyze the list of courses for all programs. Let's use pandas to help us with that:

In [17]:
courses_df = pd.DataFrame(index=epfl_courses[0], data=epfl_courses[1])

In [18]:
courses_df.columns

Index(['name', 'section', 'language', 'note', 'path', 'semester', 'exam_form',
       'coefficient', 'subject_examined', 'lecture', 'exercises', 'lecturers',
       'required', 'recommended', 'concepts', 'in_the_programs',
       'prerequisite_for', 'project', 'number_of_places', 'practical_work',
       'credits', '', 'labs'],
      dtype='object')

We observe there are a couple of fields we don't recognize, `coefficient` and `''`. Let's analyze those first.

In [19]:
# the '' column has all values either as NaN or '', so we can safely delete the column
courses_df[''].unique()

array([nan, ''], dtype=object)

In [20]:
courses_df.drop('', axis=1, inplace=True)

We observed that the _five_ rows having `''` field as the empty string are exactly the ones that don't have the number of credits listed, but we shall fix this soon (corresponding html element is empty).

The `coefficient` column is an alias for `credits`

In [21]:
len(courses_df)

3442

In [22]:
courses_df.coefficient.isna().sum()

3204

In [23]:
courses_df.credits.isna().sum()

243

In [24]:
(courses_df.credits == courses_df.coefficient).any()

False

No rows have both `credits` and `coefficient`, and there are 5 rows which have neither. Let's first merge the columns and then manually fill in the 5 gaps if possible

In [25]:
courses_df.credits.fillna(courses_df.coefficient, inplace=True)
courses_df.drop('coefficient', axis=1, inplace=True)

Let's now see the reason behind the 5 gaps:

In [26]:
courses_df[courses_df.credits.isna()].index

Index(['ENG-274', 'CH-361', 'CH-361', 'PENS-200', 'PHYS-300(a)'], dtype='object')

* ENG-274 is without credits
* CH-361 is an optional course, pointing to one of the advanced general chemistry courses
* PENS-200 Ground control in Swiss law, credits are included in the ENAC week
* PHYS-300(a) is also without credits

<a id='epfl-courses-discrepancies'></a>
We know that courses are repeated in the dataframe (row indices), but we need to check if the information is the same when course codes match. It is likely that they are the same, but we can't be sure because of the way the coursebooks were harvested (using program-specific query string parameters for fields in the right column of the page).

In [27]:
f"{courses_df.index.nunique()} unique courses and {len(courses_df)} courses with duplicates"

'1885 unique courses and 3442 courses with duplicates'

Let's see if we have a unique row count of 1 for each course code:

In [28]:
# Convert list columns to tuple to avoid this error when calling nunique():
# TypeError: unhashable type: 'list'
list_columns = ['lecturers', 'required', 'recommended', 'concepts', 'prerequisite_for', 'in_the_programs']

In [29]:
courses_df[list_columns] = courses_df[list_columns].applymap(tuple)
# lecturers and in_the_programs field is a list of lists
courses_df['lecturers'] = courses_df.lecturers.apply(lambda x: tuple(map(tuple, x)))
courses_df['in_the_programs'] = courses_df.in_the_programs.apply(lambda x: tuple(map(tuple, x)))

In [30]:
# path is always different 
group = courses_df.drop('path', axis=1).groupby(courses_df.index)

In [31]:
group.nunique().loc[(group.nunique() > 1).any(axis=1)]

,name,section,language,note,semester,exam_form,subject_examined,lecture,exercises,lecturers,required,recommended,concepts,in_the_programs,prerequisite_for,project,number_of_places,practical_work,credits,labs
BIO-502,1,1,1,1,1,1,1,0,0,1,1,1,1,2,1,0,0,1,1,0
BIOENG-404,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,0,0,0,1,0
CH-361,1,1,1,2,1,1,2,1,1,1,1,1,1,2,1,0,0,0,0,0
COM-208,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,0,0,0,1,0
COM-480,1,1,1,1,2,1,1,1,0,1,1,1,1,1,1,1,0,0,1,0
CS-401,1,1,1,1,1,1,1,1,0,1,1,1,1,2,1,1,0,0,1,0
CS-433,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,0
EE-111,1,1,1,2,2,1,1,1,1,1,1,1,1,1,1,0,0,0,1,0
EE-492(d),1,1,1,2,1,1,1,0,0,1,1,1,1,2,1,1,0,0,1,0
EE-727,1,1,1,1,2,1,1,1,0,1,1,1,1,1,1,0,1,1,1,0


There are very few exceptions that will require manual overview, and we will handle them later. Let's just get right into creating the processed `epfl_courses` dictionary and save it to a json file.

In [32]:
courses_unique_df = courses_df.loc[~courses_df.index.duplicated()]

In [33]:
len(courses_unique_df)

1885

In [34]:
epfl_courses = courses_unique_df.to_dict(orient='index')

In [35]:
epfl_courses["AR-126"]

{'name': 'Building technology II',
 'section': 'AR',
 'language': 'French',
 'note': '',
 'path': '/coursebook/en/building-technology-ii-AR-126?cb_cycle=bama_prop&cb_section=ar',
 'semester': 'Spring',
 'exam_form': 'Oral',
 'subject_examined': 'Building technology II',
 'lecture': '3 Hour(s) per week x 12 weeks',
 'exercises': '2 Hour(s) per week x 12 weeks',
 'lecturers': (('Tombesi Paolo', 'http://people.epfl.ch/223776'),
  ('Zurbrügg Peter', 'http://people.epfl.ch/104348')),
 'required': ('Technologie du bâti I (AR-125)Physique du bâtiment (PHYS-118)Structures I (CIVIL-122)',),
 'recommended': (),
 'concepts': (),
 'in_the_programs': (('Architecture', 'Bachelor semester 2'),),
 'prerequisite_for': ('Technologie du bâti III',),
 'project': nan,
 'number_of_places': nan,
 'practical_work': nan,
 'credits': '5',
 'labs': nan}

In [36]:
# storing the dict in a json file
with open('../data/processed/epfl_courses.json', 'w') as json_file:
    json.dump(epfl_courses, json_file, indent=4)

Create `epfl_programs`

In [37]:
# use dict comprehension to produce epfl_programs
epfl_programs = {
    level_name: {
        program_name: list(program.keys())
        for program_name, program in level.items()
    }
    for level_name, level in epfl.items()
}

#storing the dict in a json file
with open('../data/processed/epfl_programs.json', 'w') as json_file:
    json.dump(epfl_programs, json_file, indent=4)

In [38]:
epfl_programs['bachelor'].keys()

dict_keys(['Architecture', 'Chemistry and Chemical Engineering', 'Civil Engineering', 'Communication Systems', 'Computer Science', 'Design Together ENAC', 'Electrical and Electronics Engineering', 'Environmental Sciences and Engineering', 'Humanities and Social Sciences Program', 'Life Sciences Engineering', 'Materials Science and Engineering', 'Mathematics', 'Mechanical Engineering', 'Microengineering', 'Physics'])

In [39]:
epfl_programs['bachelor']['Life Sciences Engineering'][:10]

['MATH-203(a)',
 'MATH-207(a)',
 'PHYS-201(a)',
 'PHYS-207(a)',
 'MATH-251(c)',
 'MATH-231',
 'MATH-236',
 'BIO-212',
 'BIO-213',
 'BIO-205']

Next step (goal for next milestone) is to process the registration data and integrate it with `epfl_courses`

In [40]:
import json
with open("../data/raw/isa-registrations.json") as file:
    isa = json.load(file)

In [41]:
isa['Data visualization']

{'2019-2020': {'Digital Humanities': {'count': '6', 'level': 'master'},
  'Electrical and Electronics Engineering': {'count': '9', 'level': 'master'},
  'Computer Science': {'count': '66', 'level': 'master'},
  'Computer Science - Cybersecurity': {'count': '4', 'level': 'master'},
  'Data science minor': {'count': '14', 'level': 'minor'},
  'Data Science': {'count': '49', 'level': 'master'},
  'Communication Systems - master program': {'count': '6', 'level': 'master'}},
 '2018-2019': {'Data science minor': {'count': '2', 'level': 'minor'},
  'Computer Science': {'count': '66', 'level': 'master'},
  'Digital Humanities': {'count': '1', 'level': 'master'},
  'Electrical Engineering (edoc)': {'count': '4', 'level': 'doctoral_school'},
  'Data Science': {'count': '26', 'level': 'master'},
  'Communication Systems - master program': {'count': '11', 'level': 'master'},
  'Electrical and Electronics Engineering': {'count': '2', 'level': 'master'}},
 '2017-2018': {'Communication Systems - mast